In [1]:
import json
import time
from functools import wraps

import geopandas as gpd
import numpy as np
from geodatasets import get_path

from libpysal import __version__
from libpysal.graph import Graph

__version__

'4.13.1.dev14+ga01ec777'

Timer decorator

In [2]:
def timer(iterations):
    """Decorator to time function execution."""

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            times = []
            for _ in range(iterations):
                start_time = time.perf_counter()
                _ = func(*args, **kwargs)
                end_time = time.perf_counter()
                times.append(end_time - start_time)
            return {'mean': float(np.mean(times)), 'std': float(np.std(times))}

        return wrapper
    return decorator

Test data

In [3]:
gdf = gpd.read_file(get_path('geoda south'))
gdf_name = gdf.set_index(gdf.NAME + " " + gdf.STATE_NAME)
gdf_points = gdf.set_geometry(gdf.representative_point())
gdf_name_points = gdf_name.set_geometry(gdf_name.representative_point())

small_int = Graph.build_knn(gdf_points, k=10)
large_int = Graph.build_knn(gdf_points, k=500)
small_str = Graph.build_knn(gdf_name_points, k=10)
large_str = Graph.build_knn(gdf_name_points, k=500)

Timer functions

In [4]:
@timer(10)
def queen(gdf, strict=False):
    Graph.build_contiguity(gdf, strict=strict)


@timer(10)
def knn(gdf, k):
    Graph.build_knn(gdf, k=k)

@timer(5)
def kernel(gdf):
    Graph.build_kernel(gdf)

@timer(10)
def assign_self_weight(graph):
    graph.assign_self_weight()

@timer(1000)
def sparse(graph):
    _ = graph.sparse

@timer(10)
def subgraph(graph, ids):
    graph.subgraph(ids)

Run benchmarks

In [5]:
results = {'version': __version__}

results['queen_int'] = queen(gdf)
results['queen_str'] = queen(gdf_name)
results['queen_int_strict'] = queen(gdf, strict=True)
results['queen_str_strict'] = queen(gdf_name, strict=True)
results['knn_10_int'] = knn(gdf_points, k=10)
results['knn_10_str'] = knn(gdf_name_points, k=10)
results['knn_500_int'] = knn(gdf_points, k=500)
results['knn_500_str'] = knn(gdf_name_points, k=500)
results['kernel_int'] = kernel(gdf_points)
results['kernel_str'] = kernel(gdf_name_points)
results['assign_self_weight_small_int'] = assign_self_weight(small_int)
results['assign_self_weight_large_int'] = assign_self_weight(large_int)
results['assign_self_weight_small_str'] = assign_self_weight(small_str)
results['assign_self_weight_large_str'] = assign_self_weight(large_str)
results['sparse_small_int'] = sparse(small_int)
results['sparse_large_int'] = sparse(large_int)
results['sparse_small_str'] = sparse(small_str)
results['sparse_large_str'] = sparse(large_str)
results['subgraph_small_int'] = subgraph(small_int, gdf.index.to_series().sample(gdf.shape[0] // 5).values)
results['subgraph_large_int'] = subgraph(large_int, gdf.index.to_series().sample(gdf.shape[0] // 5).values)
results['subgraph_small_str'] = subgraph(small_str, gdf_name.index.to_series().sample(gdf.shape[0] // 5).values)
results['subgraph_large_str'] = subgraph(large_str, gdf_name.index.to_series().sample(gdf.shape[0] // 5).values)

Save to file

In [6]:
with open('bench.json', 'w') as f:
    json.dump(results, f, indent=4)